In [1]:
import pandas as pd
import sys
import os
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
SAMPLES = [30026, 30104, 30526, 30611, 30918, 30939, 30948, 31195, 31280, 31299]
CAMISIMDIR = '/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/CAMISIMGraphQuality/camisim_reads/'
TAXADIR = '/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/'

aln_files = {s: glob.glob(os.path.join(TAXADIR, f'{s}_20M.blast.txt')) for s in SAMPLES}
aln_files

{30026: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/30026_20M.blast.txt'],
 30104: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/30104_20M.blast.txt'],
 30526: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/30526_20M.blast.txt'],
 30611: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/30611_20M.blast.txt'],
 30918: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/30918_20M.blast.txt'],
 30939: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/30939_20M.blast.txt'],
 30948: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/30948_20M.blast.txt'],
 31195: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/31195_20M.blast.txt'],
 31280: ['/burg/pmg/users/ic2465/Projects/MANU_copangraph/data/TaxonomicResolution/31280_20M.blast.txt'],
 31299: ['/burg/pmg/users/ic2465/Projects/MANU

In [14]:

def genome_to_id_tab(s):
    df = pd.read_csv(open(os.path.join(CAMISIMDIR, f'{s}_out/genome_to_id.tsv')), sep='\t', header=None)
    df.columns = ['genome_id', 'file']
    df.file = df.file.apply(lambda x: os.path.basename(x))
    df.index = df.file
    return df

def get_subject_table(fa, g2id):
    file = os.path.basename(fa)
    with open(fa) as f:
        # get all headers
        headers = [l for l in f if l.startswith('>')]
    subjects = [(e.split(' ')[0][1:],file, g2id.loc[file, 'genome_id']) for e in headers]
    return pd.DataFrame(subjects, columns=['subject', 'genome', 'genome_id'])

def parse_taxonomic_profile(s):
    prof = os.path.join(CAMISIMDIR, f'{s}_out/taxonomic_profile_0.txt')
    with open(prof) as f:
        lines = [e.strip() for e in f]
        lines = lines[5:]
        lines = [tuple(e.split('\t')) for e in lines]
    profile = pd.DataFrame(lines, columns=['taxid', 'taxa_rank', 'taxpath', 'taxpathsn', 'percentage', 'genome_id', 'cami_otu'])
    profile = profile.loc[profile.taxa_rank == 'strain', :]
    profile['species'] = profile.taxpathsn.apply(lambda x: x.split('|')[6])
    profile['genera'] = profile.taxpathsn.apply(lambda x: x.split('|')[5])
    profile['family'] = profile.taxpathsn.apply(lambda x: x.split('|')[4])
    profile = profile[['taxid', 'taxa_rank', 'percentage', 'genome_id', 'species', 'genera', 'family']]
    return profile



In [15]:
all_alns = list()
for s in SAMPLES:
    print(s)
    profile = parse_taxonomic_profile(s)
    fastas = glob.glob(os.path.join(CAMISIMDIR, f'{s}_out/genomes/GCA*.fa'))
    g2id = genome_to_id_tab(s)
    fastas = [e for e in fastas if os.path.basename(e) in g2id.index]
    subjects = pd.concat([get_subject_table(e, g2id) for e in fastas])
    subjects = pd.merge(subjects, profile, on='genome_id')
    alns = pd.read_csv(os.path.join(TAXADIR, f'{s}_20M.blast.txt'), sep='\t', header=None)
    alns.columns = ['contig_name', 'subject', 'identity', 'aln_len', 'query_len']
    alns = pd.merge(alns, subjects, on='subject')
    alns['key'] = alns.contig_name.apply(lambda x: f'{s}:{x}')
    alns = alns.loc[((alns.aln_len / alns.query_len) > 0.98) & ((alns.aln_len/alns.query_len) < 1.02) & (alns.identity >= 98), :]
    alns['sample'] = s
    all_alns.append(alns)

all_alns = pd.concat(all_alns).reset_index(drop=True)
all_alns = all_alns.drop_duplicates()
all_alns


30026
30104


KeyboardInterrupt: 

In [4]:
# parse copangraph
sds = ['00', '001', '005', '01', '02', '03', '04', '05', '075', '10', '15', '20', '25', '30']
alpha_divs= list()
for sd in sds:
    print(sd)
    with open(os.path.join(TAXADIR, f'graph_sd{sd}.fasta')) as f:
        data = [e.strip().split(':') for e in f if e.startswith('>')]
        data = [(node[1:], sample[:-7], contig) for node, _, sample, _, contig, _, _, _ in data]

    copan_seqs = pd.DataFrame(data, columns=['node', 'sample' ,'contig_name'])
    copan_seqs['key'] = copan_seqs.apply(lambda x: x['sample'] + ':' + x['contig_name'], axis=1)
    copan_alns = pd.merge(copan_seqs[['node', 'key']], all_alns, on='key')
    alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))
    alphas['sd'] = sd
    alpha_divs.append(alphas)
alpha_divs = pd.concat(alpha_divs)


00


/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: RuntimeWarning: divide by zero encountered in log
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))
/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))


001


/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: RuntimeWarning: divide by zero encountered in log
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))
/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))


005


...

/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: RuntimeWarning: divide by zero encountered in log
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))
/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))


25


/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: RuntimeWarning: divide by zero encountered in log
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))
/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))


30


/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: RuntimeWarning: divide by zero encountered in log
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))
/burg/pmg/users/ic2465/tmp/ipykernel_2739164/3283136187.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  alphas = copan_alns.groupby('node').apply(lambda x: pd.Series([np.log(x.species.nunique()), np.log(x.genera.nunique()), np.log(x.family.nunique())], index=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm']))


In [5]:
#alpha_divs.to_csv(os.path.join(TAXADIR, 'alpha_divs.csv'))
alpha_divs = pd.read_csv(os.path.join(TAXADIR, 'alpha_divs.csv'))

In [3]:
#all_alns.to_csv(os.path.join(TAXADIR, 'all_alns.csv'))
all_alns = pd.read_csv(os.path.join(TAXADIR, 'all_alns.csv'), index_col=0)
all_alns

,contig_name,subject,identity,aln_len,query_len,genome,genome_id,taxid,taxa_rank,percentage,species,genera,family,key,sample
0,k141_1240,AP008937.1,100.000,211,211,GCA_000010145.1_ASM1014v1.fa,Genome3.0,334390.1,strain,0.9545,Lactobacillus fermentum,Lactobacillus,Lactobacillaceae,30026:k141_1240,30026
2,k141_0,HE970764.1_2,99.745,392,392,GCA_000318035.1_ASM31803v1.fa,Genome12.0,1215914.1,strain,0.0543,Lactobacillus casei,Lactobacillus,Lactobacillaceae,30026:k141_0,30026
3,k141_2480,CP003799.1,99.694,327,326,GCA_000165775.3_ASM16577v3.fa,Genome4.0,880633.1,strain,0.8790,Lactobacillus helveticus,Lactobacillus,Lactobacillaceae,30026:k141_2480,30026
4,k141_2480,CP003799.1,99.387,326,326,GCA_000165775.3_ASM16577v3.fa,Genome4.0,880633.1,strain,0.8790,Lactobacillus helveticus,Lactobacillus,Lactobacillaceae,30026:k141_2480,30026
7,k141_2480,CP003799.1_3,98.165,327,326,GCA_000165775.3_ASM16577v3.fa,Genome4.0,880633.1,strain,0.8790,Lactobacillus helveticus,Lactobacillus,Lactobacillaceae,30026:k141_2480,30026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090743,k141_10856,CP001810.1_2,99.520,417,417,GCA_000145035.1_ASM14503v1.fa,Genome30.0,515622.1,strain,0.0898,Butyrivibrio proteoclasticus,Butyrivibrio,Lachnospiraceae,31299:k141_10856,31299
1090751,k141_10857,CP000728.1,99.978,4499,4499,GCA_000017065.1_ASM1706v1.fa,Genome18.0,441772.1,strain,0.5587,Clostridium botulinum,Clostridium,Clostridiaceae,31299:k141_10857,31299
1090755,k141_10858,CP018623.1,99.688,321,321,GCA_001908255.1_ASM190825v1.fa,Genome24.0,1311.1,strain,0.2073,Streptococcus agalactiae,Streptococcus,Streptococcaceae,31299:k141_10858,31299
1090756,k141_10859,CP003939.1,99.961,2551,2551,GCA_001010825.1_ASM101082v1.fa,Genome20.0,1224745.1,strain,0.4154,Clostridioides difficile,Clostridioides,Peptostreptococcaceae,31299:k141_10859,31299


In [5]:
plot_dat = pd.melt(alpha_divs.reset_index(), value_vars=['alpha_div_sp', 'alpha_div_gn', 'alpha_div_fm'], id_vars=['node', 'sd'])
plot_dat.sd = plot_dat.sd.apply(lambda x: float(f'0.{x}'))

In [25]:
plot_dat.loc[(plot_dat.sd == 0.1) & (plot_dat.variable == 'alpha_div_fm'),:]['value'].replace(-np.inf, np.nan).dropna().std()

0.04372653084981931

In [16]:
def plot_unlabelled_version(ax, name, tight_layout=True):
    name = os.path.splitext(name)[0]
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    frame1 = plt.gca()
    frame1.legend().set_visible(False)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(f'{name}_UNLBLD.pdf', dpi=1400, bbox_inches='tight')
    plt.savefig(f'{name}_UNLBLD.png', dpi=900, bbox_inches='tight')
    plt.clf()

In [17]:
plt.figure(figsize=(4,4))
ax = sns.lineplot(x=plot_dat.sd, y=plot_dat.value, hue=plot_dat.variable)
plt.yscale('log')
name = os.path.join(TAXADIR, 'taxonomy_curve.pdf')
ax.set_ylabel('alpha-diversity')
plt.savefig(name, dpi=1400, bbox_inches='tight')
plot_unlabelled_version(ax, name)

<Figure size 400x400 with 0 Axes>

In [ ]:
kspecies_alpha.describe()

count    222914.000000
mean          0.004425
std           0.059882
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           2.302585
dtype: float64

In [64]:
np.log10(10)

1.0